In [87]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd

In [88]:
# URL of page to be scraped
url = 'https://redplanetscience.com'
space_image = "https://spaceimages-mars.com"
mars_facts = "https://galaxyfacts-mars.com"
hemisph = "https://marshemispheres.com"

In [105]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/Galya/.wdm/drivers/chromedriver/mac64/93.0.4577.15/chromedriver] found in cache


In [ ]:
browser.visit(url)

In [ ]:
html = browser.html

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Examine the results, then determine element that contains news info
print(soup.prettify())

### NASA Mars News

In [ ]:
# Find the latest news
latest_news = soup.find('div', id='news')
latest_news

In [ ]:
# The latest news title
news_title = latest_news.find('div', class_='content_title').text
news_title

In [ ]:
# The latest news paragraph
news_p = latest_news.find('div', class_='article_teaser_body').text
news_p

### JPL Mars Space Images - Featured Image

In [ ]:
# Visit the url for the Featured Space Image site 
browser.visit(space_image)

In [ ]:
html_img = browser.html

In [ ]:
soup_img = BeautifulSoup(html_img, 'html.parser')

In [ ]:
print(soup_img.prettify())

In [ ]:
local_image_url = soup_img.find('img', class_="headerimage fade-in")['src']
local_image_url

In [ ]:
featured_image_url = space_image+'/'+local_image_url
featured_image_url

### Mars Facts

In [ ]:
tables = pd.read_html(mars_facts)
tables

In [ ]:
df = tables[0]
df

In [ ]:
df.rename(columns = {0:'Facts', 1:'Mars', 2:'Earth'}, inplace=True)
df

In [ ]:
df.drop(0, inplace=True)
df

In [ ]:
df.set_index('Facts', inplace=True)
df

In [ ]:
html_table = df.to_html()
html_table.replace('\n', '')
html_table

### Mars Hemispheres

In [106]:
browser.visit(hemisph)

In [107]:
# Find links to full size images of mars hemispheres
hem_html = browser.html
soup_hem = BeautifulSoup(hem_html, 'html.parser')

links = []
links_text = []
items = soup_hem.find_all('div', class_='item')
for it in items:
    descr = it.find('div', class_='description')
    links_text.append(descr.find('h3').text)
    links.append(hemisph+'/'+it.find('a')['href'])
links_text

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [92]:
hem_img_ttl_urls = [
#     {"title": "Valles Marineris Hemisphere", "img_url": "..."},
#     {"title": "Cerberus Hemisphere", "img_url": "..."},
#     {"title": "Schiaparelli Hemisphere", "img_url": "..."},
#     {"title": "Syrtis Major Hemisphere", "img_url": "..."},
]


In [93]:
test = hemisph+'/'+links[0]
test

'https://marshemispheres.com/cerberus.html'

In [111]:
# browser.links.find_by_partial_href(links[0]).first.click()
# browser.click_link_by_partial_href(links[0])

title = links_text[0].replace(" Enhanced","")
title





# browser.links.find_by_partial_text(links_text[0]).click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

img_url = hemisph+'/'+soup.find('a', text = 'Original')['href']
img_url

hem_img_ttl_urls.append({"title": title, "img_url": img_url})
hem_img_ttl_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/cerberus_enhanced.tif'}]

In [115]:
# Define list to add 
hem_img_ttl_urls = []

for txt in links_text:
    
    # Save title of the hemisphere  
    title = txt.replace(" Enhanced","")
    
    # Click hemisphere to access link to the original image
    browser.links.find_by_partial_text(txt).click()
    
    # Use soup to find the link
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    img_url = hemisph+'/'+soup.find('a', text = 'Original')['href']
    
    # Add to the list of dictionaries
    hem_img_ttl_urls.append({"title": title, "img_url": img_url})
    
    # Return to the main page
    browser.links.find_by_partial_text("Back").click()

In [116]:
hem_img_ttl_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/cerberus_enhanced.tif'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/cerberus_enhanced.tif'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/cerberus_enhanced.tif'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/cerberus_enhanced.tif'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/cerberus_enhanced.tif'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced.tif

In [103]:
browser.quit()